In [2]:
import pandas as pd
pd.set_option('display.max_columns', 20)

In [19]:
df = pd.read_csv('datos/productos-gemini.csv')

In [20]:
from es_ecommerce_classifier import load as load_model

In [6]:
model = load_model()

# Test queries to demonstrate the model's capabilities
test_queries = [
    "Quiero una notebook para gaming",
    "Necesito un monitor para diseño",
    "Busco una computadora para trabajar en la oficina"
]

In [ ]:
query = test_queries[0]
doc = model(query)

# Get predictions
raw_predictions = dict(doc.cats)
raw_predictions

In [25]:
import re
import ast
import pandas as pd

df["atributos_list"] = df["atributos_list"].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else [])

def clean_label(label):
    """Clean up duplicate prefixes in model labels"""
    # Remove duplicated prefixes: CAT_CAT_, INT_INT_, ATTR_ATTR_
    # Keep only the first occurrence of the prefix
    
    # Match patterns like CAT_CAT_, INT_INT_, ATTR_ATTR_ and replace with single prefix
    cleaned = re.sub(r'^(CAT_CAT|INT_INT|ATTR_ATTR)_(.*)$', r'\1_\2', label)
    
    # If pattern doesn't match, return original label
    if cleaned == label:
        # Try to remove the first duplicate prefix manually
        parts = label.split('_')
        if len(parts) >= 3:
            # Check if first two parts are identical (e.g., CAT_CAT)
            if parts[0] == parts[1]:
                cleaned = '_'.join([parts[0]] + parts[2:])
    
    return cleaned

def similitud_producto(row, tags):
    total = 0
    if row["categoria_detectada"] in tags:
        total += 1
    if row["intencion_detectada"] in tags:
        total += 1
    total += len(set(row["atributos_list"]).intersection(tags))
    return total

def generar_tags(query):
    doc = model(query)

    # Get predictions
    raw_predictions = dict(doc.cats)
    cleaned_predictions = {}
    for raw_label, score in raw_predictions.items():
        clean_label_name = clean_label(raw_label)
        cleaned_predictions[clean_label_name] = score

    predicciones_ordenadas = sorted(cleaned_predictions.items(), key=lambda x: x[-1], reverse=True)
    tags = list(c[0] for c in predicciones_ordenadas)[:15]

    return tags

my_query = 'quiero un mouse gaming'
tags = generar_tags(my_query)

df["similitud"] = df.apply(lambda r: similitud_producto(r, tags), axis=1)

# Filtrar por productos que comparten al menos 2 tags (ajustá el umbral)
df_filtrado = df[df["similitud"] >= 2].sort_values("similitud", ascending=False)

df_filtrado.head()

,title,slug,brand_name,categories,list_price,product_specifications,product_attributes,atributos_correctos,categoria_detectada,intencion_detectada,atributos_list,similitud
278,"Tablet Lenovo 11,5” 128GB Tab Plus Luna Grey +...",tablet-lenovo-11-5”-128gb-tab-plus-luna-grey-s...,Lenovo,Tablets,699999.0,"[{'Wi-Fi': 'Wi-Fi® 5, 802.11ac 1x1'}, {'GPS': ...",[{'Incluye': '<p>Sleeve + Moto Buds (TB351FU)<...,"{'categoria': 'CAT_MONITOR', 'intencion': 'INT...",CAT_MONITOR,INT_GAMING,[],2
279,"Tablet Lenovo 11,5” 128GB Tab Plus Luna Grey +...",tablet-lenovo-11-5”-128gb-tab-plus-luna-grey-s...,Lenovo,Tablets,699999.0,"[{'Wi-Fi': 'Wi-Fi® 5, 802.11ac 1x1'}, {'GPS': ...","[{'Incluye': '<p><span style=""background-color...","{'categoria': 'CAT_MONITOR', 'intencion': 'INT...",CAT_MONITOR,INT_GAMING,[],2
282,Tablet Lenovo IdeaTab Pro 8GB 256GB Teclado+ P...,tablet-lenovo-ideatab-pro-8gb-256gb-teclado-pe...,Lenovo,Tablets,1103999.0,"[{'Wi-Fi': '802.11 a/b/g/n/ac/ax, WiFi de dobl...",[{'Video': 'https://www.youtube.com/watch?v=t1...,"{'categoria': 'CAT_MONITOR', 'intencion': 'INT...",CAT_MONITOR,INT_GAMING,[],2


In [40]:
# --- Recomendador por tags (funciona con LLM o dict de scores) ---

import re
import ast
import pandas as pd
from unicodedata import normalize as uni_normalize

# ---------------------------------
# 1) Helpers de normalización/parseo
# ---------------------------------
def safe_list(x):
    """
    Convierte strings tipo "['ATTR_X','ATTR_Y']" a lista real.
    Si ya es lista, la devuelve; si falla, devuelve [].
    """
    if isinstance(x, list):
        return x
    if isinstance(x, str):
        try:
            return ast.literal_eval(x)
        except Exception:
            return []
    return []

def clean_label(label: str) -> str:
    """
    Normaliza labels del modelo:
    - Mayúsculas
    - Sin acentos
    - Espacios/guiones -> _
    - Colapsa prefijos duplicados (CAT_CAT_ -> CAT_, INT_INT_ -> INT_, ATTR_ATTR_ -> ATTR_)
    - Colapsa underscores repetidos
    """
    if not isinstance(label, str):
        label = str(label)
    s = label.strip().upper()
    s = uni_normalize("NFKD", s).encode("ascii", "ignore").decode("ascii")
    s = re.sub(r"[ \-./:;,]+", "_", s)            # separadores -> _
    s = re.sub(r"^(CAT_)+", "CAT_", s)            # CAT_CAT_... -> CAT_
    s = re.sub(r"^(INT_)+", "INT_", s)            # INT_INT_... -> INT_
    s = re.sub(r"^(ATTR_)+", "ATTR_", s)          # ATTR_ATTR_... -> ATTR_
    s = re.sub(r"_+", "_", s)                     # varios __ -> _
    s = s.strip("_")
    return s

# ---------------------------------
# 2) Similitud (conteo de coincidencias)
# ---------------------------------
def similitud_producto(row: pd.Series, tags_set: set) -> int:
    """
    Suma:
      +1 si coincide categoria_detectada
      +1 si coincide intencion_detectada
      +1 por cada ATTR_* presente en atributos_list que esté en tags_set
    """
    total = 0
    cat = row.get("categoria_detectada", "")
    intent = row.get("intencion_detectada", "")
    attrs = row.get("atributos_list", [])

    if isinstance(cat, str) and cat in tags_set:
        total += 1
    if isinstance(intent, str) and intent in tags_set:
        total += 1
    if isinstance(attrs, list):
        total += len(set(attrs).intersection(tags_set))

    return total

# ---------------------------------
# 3) Generación de tags desde LLM o dict de scores
# ---------------------------------
def generar_tags(query: str = "",
                 model=None,
                 scores_dict: dict | None = None,
                 topn: int = 15) -> list[str]:
    """
    Devuelve la lista ordenada de tags más probables.
    - Si pasás scores_dict (p.ej. salida ya parseada de tu LLM): usa eso.
    - Si pasás model (spacy/llm con doc.cats): usa model(query).
    - Si pasás ambos, prioriza scores_dict.
    """
    if scores_dict is not None:
        raw_predictions = scores_dict
    else:
        if model is None:
            raise ValueError("Debes pasar `scores_dict` o `model` para generar tags.")
        doc = model(query)
        raw_predictions = getattr(doc, "cats", {})

    # Normalizar labels
    cleaned_predictions = { clean_label(k): float(v) for k, v in raw_predictions.items() }

    # Ordenar por score desc
    predicciones_ordenadas = sorted(cleaned_predictions.items(), key=lambda x: x[1], reverse=True)
    tags = [k for k, _ in predicciones_ordenadas[:topn]]
    return tags

# ---------------------------------
# 4) Pipeline completo
# ---------------------------------
def filtrar_por_tags(df: pd.DataFrame, tags: list[str], min_coincidencias: int = 2) -> pd.DataFrame:
    """
    Calcula similitud y devuelve DF filtrado (similitud >= min_coincidencias) y ordenado.
    """
    df = df.copy()

    # Asegurar columnas
    if "atributos_list" not in df.columns:
        df["atributos_list"] = [[] for _ in range(len(df))]
    else:
        df["atributos_list"] = df["atributos_list"].apply(safe_list)

    for col in ("categoria_detectada", "intencion_detectada"):
        if col not in df.columns:
            df[col] = ""
        df[col] = df[col].fillna("")

    # Preparar set de tags
    tags_norm = { clean_label(t) for t in tags if isinstance(t, str) and t.strip() }

    # Calcular similitud
    df["similitud"] = df.apply(lambda r: similitud_producto(r, tags_norm), axis=1)

    # Filtrar y ordenar
    df_filtrado = df[df["similitud"] >= min_coincidencias].sort_values("similitud", ascending=False)

    # Columnas útiles (ajusta según tu catálogo)
    cols_show = [c for c in [
        "title", "brand_name", "categories", "list_price",
        "categoria_detectada", "intencion_detectada", "atributos_list",
        "similitud"
    ] if c in df_filtrado.columns]

    return df_filtrado.loc[:, cols_show]

# ===========================
# ======= USO BÁSICO ========
# ===========================

# Cargar tu catálogo (si aún no lo tenés en memoria)
# df = pd.read_csv("productos-gemini.csv")

# Ejemplo de tags de referencia (como los que compartiste):
my_query = 'laptop'
tags_referencia = generar_tags(my_query, model)

# Asegúrate de haber cargado tu df antes de esto:
# df = pd.read_csv("productos-gemini.csv")

# Normalizar/parsear y filtrar
# (ajusta min_coincidencias: 1, 2, 3 según tu catálogo)
df_resultado = filtrar_por_tags(df, tags_referencia, min_coincidencias=2)

# Ver top 10
df_resultado.head(10)


,title,brand_name,categories,list_price,categoria_detectada,intencion_detectada,atributos_list,similitud
219,Notebook Lenovo Slim Procesador Intel Core I7 ...,Lenovo,Notebook,1890000.0,CAT_NOTEBOOK,INT_OFICINA,[],2
